In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc
import math
import random
import matplotlib
# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns 
import matplotlib.patches as patches
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 30)
from datetime import datetime
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
path='../input/ashrae-energy-prediction/'
path_new = '../input/lag-transform/'
path_new = '../input/transformer/'
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from math import log
import lightgbm as lgb
import datetime
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
pd.options.mode.chained_assignment = None
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import holidays
import os
from scipy.sparse import csr_matrix,coo_matrix, hstack
from sklearn.linear_model import Ridge
import seaborn as sns 

for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        print(os.path.join(dirname, filename))
def ls():
     print([v for v in globals().keys() if not v.startswith('_')])

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    
def free_space(del_list):
    for name in del_list:
        if not name.startswith('_'):
            del globals()[name]
    gc.collect()
    
def chunk_predictor(X,model,n=100,verbose=True):
    predict_fold = KFold(n)
    result = np.zeros(X.shape[0])
    if verbose:
        for trn_idx, val_idx in tqdm_notebook(predict_fold.split(X)):
               result[val_idx] = model.predict(X.iloc[val_idx,:])
    else:
        for trn_idx, val_idx in predict_fold.split(X):
            result[val_idx] = model.predict(X.iloc[val_idx,:])
    return(result)

def one_hot_encoder(train,col,tr_index):
    temp = csr_matrix(pd.get_dummies(train,prefix_sep='_',columns=col,drop_first=True))
    return(temp[:tr_index,:],temp[tr_index:,:])


../input/ashrae-energy-prediction/weather_train.csv
../input/ashrae-energy-prediction/test.csv
../input/ashrae-energy-prediction/weather_test.csv
../input/ashrae-energy-prediction/train.csv
../input/ashrae-energy-prediction/building_metadata.csv
../input/ashrae-energy-prediction/sample_submission.csv


In [2]:
path_load = '../input/train-transformer/'
train = pd.read_hdf(f'{path_load}dataset.h5',key='train')
test = pd.read_hdf(f'{path_load}dataset.h5',key='test')
# column = pd.read_hdf(f'{path_load}dataset.h5',key='column')['column'].tolist()
timestamp = pd.read_hdf(f'{path_load}dataset.h5',key='timestamp')
timestamp_test = pd.read_hdf(f'{path_load}dataset.h5',key='timestamp_test')
train['month'] = timestamp.dt.month.astype(int)
test['month'] = timestamp_test.dt.month.astype(int)
# category_column = pd.read_hdf(f'{path_load}dataset.h5',key='category_column')['cat_column'].tolist()
# target = pd.read_hdf(f'{path_load}dataset.h5',key='target')
gc.collect()

93

In [3]:
SEED = 42
seed_everything(SEED)
gc.collect()

0

In [4]:
replace = train.groupby('building_meter')['pm2k'].mean()

train.loc[train.pm2k.isnull(),'pm2k'] = train['building_meter'].map(replace)[train.pm2k.isnull()]
test.loc[test.pm2k.isnull(),'pm2k'] = test['building_meter'].map(replace)[test.pm2k.isnull()]

strat_train = train['building_meter']
strat_test = test['building_meter']

In [5]:
to_keep = ['month','day','hour','weekend','air_temperature','dew_temperature',
       'heat_index','feel','air_temperature_mean_lag24','dew_temperature_mean_lag24','feel_mean_lag24','heating_required','m2k','pm2k']
train = train[['building_meter']+to_keep]
test = test[['building_meter']+to_keep]
tr_index = train.shape[0]
all_df = train.append(test,ignore_index=True)

del train,test
gc.collect()

gc.collect()

0

In [6]:
# to_keep = ['month','day','hour','weekend','air_temperature','dew_temperature',
#        'heat_index','feel','air_temperature_mean_lag24','dew_temperature_mean_lag24','feel_mean_lag24','heating_required','m2k','pm2k']
# tr_index = train.shape[0]
# all_df = train.append(test,ignore_index=True)[['building_meter']+to_keep]
to_rescale = ['air_temperature','dew_temperature','heat_index','feel',
              'air_temperature_mean_lag24','dew_temperature_mean_lag24','feel_mean_lag24','heating_required','m2k','pm2k']

for sel in tqdm_notebook(np.sort(all_df.building_meter.unique())):
    temp = all_df.loc[all_df.building_meter==sel,to_rescale]
    m1 = temp.min()
    m2 = temp.max()
    all_df.loc[all_df.building_meter==sel,to_rescale] = (temp-m1)/(m2-m1)
    gc.collect()

train, test = all_df.iloc[:tr_index,:], all_df.iloc[tr_index:,:]
train.to_hdf('dataset_rescale.h5',key='rescale_train',complib='blosc', complevel=9)
test.to_hdf('dataset_rescale.h5',key='rescale_test',complib='blosc', complevel=9)
